In [ ]:
import os
from src.utils import data_load
import pandas as pd
from src.s3_utils import pandas_from_csv_s3
import numpy as np
from datetime import datetime, timedelta
import seaborn as sns
import matplotlib.pyplot as plt
from utils import get_survey_question, na_rate
import torch

In [ ]:
user_dict = {}
for f in os.listdir(os.path.join("/", "mnt", 'results', "edema_pred_window")):

    f_name_lst = f.split('_')
    if 'date' in f_name_lst:
        continue
    
    user_id = int(f_name_lst[1])
    if user_id in user_dict:
        user_dict[user_id] += 1
    else:
        user_dict[user_id] = 1
print(user_dict)
s = []
for i in user_dict:
    s.append(str(user_dict[i]))

sns.displot(s, shrink=.8, bins=['1', '2', '3', '4'])

In [ ]:
df = pd.DataFrame()
df['slices'] = pd.Categorical(values=s, categories=['1', '2', '3', '4']) ## Set the order for the column as you want
df.sort_values(['slices'], inplace=True) ## Sort the column
sns.displot(data = df, x = 'slices')  ## and plot

In [ ]:
len(s)